## MOBILENET ARCHITECTURE

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow
import tensorflow as tf
print(tf.__version__)

import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU



from time import time
import numpy as np

from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('ignore')


2.15.0


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split = 0.2)
train_data=train.flow_from_directory(directory = 'DATASET/TRAIN',target_size=(224,224),
                                     batch_size=32,class_mode='categorical')

Found 3462 images belonging to 36 classes.


In [4]:
test=ImageDataGenerator(rescale=1./255)
test_data=test.flow_from_directory(directory = 'DATASET/TEST',target_size=(224,224),
                                   batch_size=32,class_mode='categorical')

Found 791 images belonging to 36 classes.


In [5]:
def mobilenet(input_shape, n_classes):
  
  def mobilenet_block(x, f, s=1):
    x = DepthwiseConv2D(3, strides=s, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(f, 1, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
    
    
  input = Input(input_shape)

  x = Conv2D(32, 3, strides=2, padding='same')(input)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  x = mobilenet_block(x, 64)
  x = mobilenet_block(x, 128, 2)
  x = mobilenet_block(x, 128)

  x = mobilenet_block(x, 256, 2)
  x = mobilenet_block(x, 256)

  x = mobilenet_block(x, 512, 2)
  for _ in range(5):
    x = mobilenet_block(x, 512)

  x = mobilenet_block(x, 1024, 2)
  x = mobilenet_block(x, 1024)
  
  x = GlobalAvgPool2D()(x)
  
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy',tensorflow.keras.metrics.Precision()])
  return model
     

input_shape = 224, 224, 3
n_classes = 36

K.clear_session()
model = mobilenet(input_shape, n_classes)
model.summary() 




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 112, 112, 32)      128       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 112, 112, 32)      0         
                                                                 
 depthwise_conv2d (Depthwis  (None, 112, 112, 32)      320       
 eConv2D)                                                        
                                                                 
 batch_normalization_1 (Bat  (None, 112, 112, 32)      128

 re_lu_14 (ReLU)             (None, 14, 14, 512)       0         
                                                                 
 depthwise_conv2d_7 (Depthw  (None, 14, 14, 512)       5120      
 iseConv2D)                                                      
                                                                 
 batch_normalization_15 (Ba  (None, 14, 14, 512)       2048      
 tchNormalization)                                               
                                                                 
 re_lu_15 (ReLU)             (None, 14, 14, 512)       0         
                                                                 
 conv2d_8 (Conv2D)           (None, 14, 14, 512)       262656    
                                                                 
 batch_normalization_16 (Ba  (None, 14, 14, 512)       2048      
 tchNormalization)                                               
                                                                 
 re_lu_16 

In [6]:
model_path = "MOBILENET.h5"

from keras.callbacks import ModelCheckpoint

M = ModelCheckpoint(model_path, monitor='accuracy', verbose=1, save_best_only=True)

In [7]:
epochs = 70
batch_size = 64

In [8]:
#### Fitting the model
history = model.fit(
           train_data, steps_per_epoch=train_data.samples // batch_size, 
           epochs=epochs, 
#            validation_data=test_data,validation_steps=test_data.samples // batch_size,
           callbacks=[M])

Epoch 1/100


54/54 [==============================] - ETA: 0s - loss: 3.5062 - accuracy: 0.0856 - precision: 0.3191
Epoch 1: accuracy improved from -inf to 0.08565, saving model to MOBILENET.h5
54/54 [==============================] - 254s 4s/step - loss: 3.5062 - accuracy: 0.0856 - precision: 0.3191
Epoch 2/100
54/54 [==============================] - ETA: 0s - loss: 3.0506 - accuracy: 0.1516 - precision: 0.4400
Epoch 2: accuracy improved from 0.08565 to 0.15162, saving model to MOBILENET.h5
54/54 [==============================] - 227s 4s/step - loss: 3.0506 - accuracy: 0.1516 - precision: 0.4400
Epoch 3/100
54/54 [==============================] - ETA: 0s - loss: 2.7738 - accuracy: 0.2118 - precision: 0.5030
Epoch 3: accuracy improved from 0.15162 to 0.21181, saving model to MOBILENET.h5
54/54 [==============================] - 223s 4s/step - loss: 2.7738 - accuracy: 0.2118 - precision: 0.5030
Epoch 4/100
54/54 [==============================] - ETA: 0s - loss: 2.5450 - accuracy: 0

Epoch 27/100
54/54 [==============================] - ETA: 0s - loss: 0.6305 - accuracy: 0.7899 - precision: 0.8661
Epoch 27: accuracy improved from 0.78067 to 0.78993, saving model to MOBILENET.h5
54/54 [==============================] - 224s 4s/step - loss: 0.6305 - accuracy: 0.7899 - precision: 0.8661
Epoch 28/100
54/54 [==============================] - ETA: 0s - loss: 0.6168 - accuracy: 0.8067 - precision: 0.8658
Epoch 28: accuracy improved from 0.78993 to 0.80671, saving model to MOBILENET.h5
54/54 [==============================] - 232s 4s/step - loss: 0.6168 - accuracy: 0.8067 - precision: 0.8658
Epoch 29/100
54/54 [==============================] - ETA: 0s - loss: 0.5708 - accuracy: 0.8171 - precision: 0.8759
Epoch 29: accuracy improved from 0.80671 to 0.81713, saving model to MOBILENET.h5
54/54 [==============================] - 234s 4s/step - loss: 0.5708 - accuracy: 0.8171 - precision: 0.8759
Epoch 30/100
54/54 [==============================] - ETA: 0s - loss: 0.5916 - acc

Epoch 56/100
54/54 [==============================] - ETA: 0s - loss: 0.2984 - accuracy: 0.9062 - precision: 0.9217
Epoch 56: accuracy did not improve from 0.91598
54/54 [==============================] - 223s 4s/step - loss: 0.2984 - accuracy: 0.9062 - precision: 0.9217
Epoch 57/100
54/54 [==============================] - ETA: 0s - loss: 0.2414 - accuracy: 0.9195 - precision: 0.9395
Epoch 57: accuracy improved from 0.91598 to 0.91951, saving model to MOBILENET.h5
54/54 [==============================] - 219s 4s/step - loss: 0.2414 - accuracy: 0.9195 - precision: 0.9395
Epoch 58/100
54/54 [==============================] - ETA: 0s - loss: 0.2620 - accuracy: 0.9101 - precision: 0.9339
Epoch 58: accuracy did not improve from 0.91951
54/54 [==============================] - 222s 4s/step - loss: 0.2620 - accuracy: 0.9101 - precision: 0.9339
Epoch 59/100
54/54 [==============================] - ETA: 0s - loss: 0.2047 - accuracy: 0.9317 - precision: 0.9507
Epoch 59: accuracy improved from 0

KeyboardInterrupt: 

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(20, 8))
plt.plot(history.history['accuracy'])

for i in range(epochs):
    if i%5 == 0:
        plt.annotate(np.round(history.history['accuracy'][i]*100,2),xy=(i,history.history['accuracy'][i]))

plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(history.history['loss'])

for i in range(epochs):
    if i%5 == 0:
        plt.annotate(np.round(history.history['loss'][i]*100,2),xy=(i,history.history['loss'][i]))

plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()